In [74]:
import pandas as pd
import numpy as np

In [75]:
dataset=pd.read_csv("D:\End to end project\Digital Blink\Emoji Predictor\emoji_cleaned.csv")
dataset.head()

,Tweet,Label
0,a little throwback with my favourite person w...,0
1,glam on yesterday for kcon makeup using in f...,7
2,democracy plaza in the wake of a stunning outc...,11
3,then amp now vilo walt disney magic kingdom,0
4,who never a galaxy far far away,2


In [76]:
map_data=open(r"D:\End to end project\Digital Blink\Emoji Predictor\us_mapping.txt",'r')
lines = map_data.readlines()  # Read lines from file
map_data.close()

In [77]:
records = [line.strip().split('\t') for line in lines]
dataset_map = pd.DataFrame(records, columns=["Score", "emoji", "name"])

dataset_map.head()

,Score,emoji,name
0,0,❤️,Red heart
1,1,😍,Smiling face with hearteyes
2,2,😂,Face with tears of joy
3,3,💕,Two hearts
4,4,🔥,Fire


In [78]:
map_dict=dataset_map.set_index('Score').T.to_dict('list')
map_dict

{'0': ['❤️', ' Red heart'],
 '1': ['😍', ' Smiling face with hearteyes'],
 '2': ['😂', ' Face with tears of joy'],
 '3': ['💕', ' Two hearts'],
 '4': ['🔥', ' Fire'],
 '5': ['😊', ' Smiling face with smiling eyes'],
 '6': ['😎', ' Smiling face with sunglasses'],
 '7': ['✨', ' Sparkles'],
 '8': ['💙', ' Blue heart'],
 '9': ['😘', ' Face blowing a kiss'],
 '10': ['📷', ' Camera'],
 '11': ['🇺🇸', ' United States'],
 '12': ['☀️', ' Sun'],
 '13': ['💜', ' Purple heart'],
 '14': ['😉', ' Winking face'],
 '15': ['💯', ' Hundred points'],
 '16': ['😁', ' Beaming face with smiling eyes'],
 '17': ['🎄', 'Christmas tree'],
 '18': ['📸', 'Camera with flash'],
 '19': ['😜', 'Winking face with tongue']}

In [79]:
type(dataset['Tweet'])

pandas.core.series.Series

In [80]:
dataset.drop(index=16070)

,Tweet,Label
0,a little throwback with my favourite person w...,0
1,glam on yesterday for kcon makeup using in f...,7
2,democracy plaza in the wake of a stunning outc...,11
3,then amp now vilo walt disney magic kingdom,0
4,who never a galaxy far far away,2
...,...,...
49995,my ootd love this chain so much and our new ri...,1
49996,met santa and olaf the north pole today nort...,0
49997,new york by night strideby herelocationnyc see...,11
49998,kisses for the birthday girl helzberg diamonds,0


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset['Tweet'] = dataset['Tweet'].astype(str)


dataset['Tweet'] = dataset['Tweet'].apply(
    lambda x: tokenizer.tokenize(x)
)


In [82]:
dataset

,Tweet,Label
0,"[a, little, throw, ##back, with, my, favourite...",0
1,"[g, ##lam, on, yesterday, for, kc, ##on, makeu...",7
2,"[democracy, plaza, in, the, wake, of, a, stunn...",11
3,"[then, amp, now, vi, ##lo, walt, disney, magic...",0
4,"[who, never, a, galaxy, far, far, away]",2
...,...,...
49995,"[my, o, ##ot, ##d, love, this, chain, so, much...",1
49996,"[met, santa, and, olaf, the, north, pole, toda...",0
49997,"[new, york, by, night, stride, ##by, here, ##l...",11
49998,"[kisses, for, the, birthday, girl, he, ##lz, #...",0


In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
w2v_model = Word2Vec(
    sentences=dataset['Tweet'],            # list of tokenized sentences
    vector_size=100,      # size of embedding vector
    window=5,             # context window
    min_count=1,          
    workers=4,            
    sg=1                  
)


In [84]:
dataset

,Tweet,Label
0,"[a, little, throw, ##back, with, my, favourite...",0
1,"[g, ##lam, on, yesterday, for, kc, ##on, makeu...",7
2,"[democracy, plaza, in, the, wake, of, a, stunn...",11
3,"[then, amp, now, vi, ##lo, walt, disney, magic...",0
4,"[who, never, a, galaxy, far, far, away]",2
...,...,...
49995,"[my, o, ##ot, ##d, love, this, chain, so, much...",1
49996,"[met, santa, and, olaf, the, north, pole, toda...",0
49997,"[new, york, by, night, stride, ##by, here, ##l...",11
49998,"[kisses, for, the, birthday, girl, he, ##lz, #...",0


In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences=dataset['Tweet'], vector_size=100, min_count=1)

def sentence_embedding(tokens):
    vecs = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    if len(vecs) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(vecs, axis=0)
dataset['sentence_vector'] = dataset['Tweet'].apply(sentence_embedding)


In [87]:
dataset

,Tweet,Label,sentence_vector
0,"[a, little, throw, ##back, with, my, favourite...",0,"[-0.05411576, 0.51391613, -0.34405714, -0.0724..."
1,"[g, ##lam, on, yesterday, for, kc, ##on, makeu...",7,"[-0.15656765, 0.18488812, 0.09924544, -0.09187..."
2,"[democracy, plaza, in, the, wake, of, a, stunn...",11,"[-0.20746422, 0.13515742, -0.11181886, -0.1534..."
3,"[then, amp, now, vi, ##lo, walt, disney, magic...",0,"[-0.7232284, 0.22566159, -0.0238593, 0.4682785..."
4,"[who, never, a, galaxy, far, far, away]",2,"[-0.11715334, 0.5868083, -0.11975988, -0.31505..."
...,...,...,...
49995,"[my, o, ##ot, ##d, love, this, chain, so, much...",1,"[0.057419136, 0.5462586, 0.050390612, -0.02191..."
49996,"[met, santa, and, olaf, the, north, pole, toda...",0,"[-0.050597295, 0.24881114, -0.19427519, -0.030..."
49997,"[new, york, by, night, stride, ##by, here, ##l...",11,"[-0.0051931962, 0.2199232, -0.04641551, 0.2292..."
49998,"[kisses, for, the, birthday, girl, he, ##lz, #...",0,"[-0.054742143, 0.15744086, 0.18865676, -0.0402..."


In [88]:
x=dataset['sentence_vector']
y=dataset['Label']

In [91]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(np.vstack(x))


In [92]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [93]:
x_train = np.vstack(x_train)
x_test = np.vstack(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout 

model = Sequential([
    Dense(512, activation='relu', input_shape=(100,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20,batch_size=128, validation_data=(x_test, y_test))


Epoch 1/20


c:\Users\ACER\miniconda3\envs\venv2\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ACER\miniconda3\envs\venv2\lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\ACER\miniconda3\envs\venv2\lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1052 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1057 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1054 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1066 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1073 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1072 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1062 - loss: 0.0000e+00 - val_accuracy: 0.1035 - val_loss: 0.0000e+00
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accur

In [95]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0991 - loss: 0.0000e+00
Test Accuracy: 0.1035


In [96]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.27      0.80      0.40      2153
           1       0.20      0.08      0.11      1035
           2       0.29      0.51      0.37      1048
           3       0.27      0.01      0.01       574
           4       0.27      0.23      0.25       520
           5       0.27      0.02      0.04       486
           6       0.11      0.01      0.02       390
           7       0.21      0.02      0.04       376
           8       0.33      0.01      0.01       382
           9       0.20      0.01      0.01       341
          10       0.15      0.05      0.07       273
          11       0.29      0.15      0.20       329
          12       0.26      0.16      0.20       286
          13       0.00      0.00      0.00       270
          14       0.00      0.00      0.00       272
          15       0.14      0.01      0.02       251
          16       0.00      0.00      0.00       246
          17       0.75    

c:\Users\ACER\miniconda3\envs\venv2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\miniconda3\envs\venv2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\miniconda3\envs\venv2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
gnb=GaussianNB()
gnb.fit(x_train,y_train)


y_pred=gnb.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.40      0.07      0.12      2153
           1       0.15      0.02      0.03      1035
           2       0.26      0.39      0.31      1048
           3       0.13      0.15      0.14       574
           4       0.16      0.12      0.13       520
           5       0.09      0.05      0.06       486
           6       0.07      0.01      0.01       390
           7       0.13      0.06      0.08       376
           8       0.07      0.02      0.03       382
           9       0.08      0.13      0.10       341
          10       0.04      0.44      0.08       273
          11       0.12      0.22      0.15       329
          12       0.13      0.27      0.17       286
          13       0.09      0.01      0.01       270
          14       0.05      0.23      0.08       272
          15       0.00      0.00      0.00       251
          16       0.04      0.02      0.03       246
          17       0.66    

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=1, decision_function_shape='ovr')  
svm_model.fit(x_train, y_train)


y_pred = svm_model.predict(x_test)
print(classification_report(y_test, y_pred))